In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("IMDb Big Data Project").getOrCreate()

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [8]:
#шлях до датасету
imdb_path = "/content/drive/MyDrive/imdb/"

# **ЗАГАЛЬНА ІНФОРМАЦІЯ ПРО НАБІР ДАНИХ**

In [12]:
name_basics_schema = StructType([
        StructField("nconst", StringType(), True),
        StructField("primaryName", StringType(), True),
        StructField("birthYear", IntegerType(), True),
        StructField("deathYear", IntegerType(), True),
        StructField("primaryProfession", StringType(), True),
        StructField("knownForTitles", StringType(), True),
    ])

title_akas_schema = StructType([
        StructField("titleId", StringType(), True),
        StructField("ordering", IntegerType(), True),
        StructField("title", StringType(), True),
        StructField("region", StringType(), True),
        StructField("language", StringType(), True),
        StructField("types", StringType(), True),
        StructField("attributes", StringType(), True),
        StructField("isOriginalTitle", IntegerType(), True),
    ])

title_schema = StructType([
        StructField("tconst", StringType(), True),
        StructField("titleType", StringType(), True),
        StructField("primaryTitle", StringType(), True),
        StructField("originalTitle", StringType(), True),
        StructField("isAdult", IntegerType(), True),
        StructField("startYear", IntegerType(), True),
        StructField("endYear", IntegerType(), True),
        StructField("runtimeMinutes", IntegerType(), True),
        StructField("genres", StringType(), True),
    ])

title_crew_schema = StructType([
        StructField("tconst", StringType(), True),
        StructField("directors", StringType(), True),
        StructField("writers", StringType(), True),
    ])

title_episode_schema = StructType([
        StructField("tconst", StringType(), True),
        StructField("parentTconst", StringType(), True),
        StructField("seasonNumber", IntegerType(), True),
        StructField("episodeNumber", IntegerType(), True),
    ])

title_principals_schema = StructType([
        StructField("tconst", StringType(), True),
        StructField("ordering", IntegerType(), True),
        StructField("nconst", StringType(), True),
        StructField("category", StringType(), True),
        StructField("job", StringType(), True),
        StructField("characters", StringType(), True),
    ])

title_ratings_schema = StructType([
        StructField("tconst", StringType(), True),
        StructField("averageRating", FloatType(), True),
        StructField("numVotes", IntegerType(), True),
    ])

df_people = spark.read.csv(imdb_path + "name.basics.tsv", sep="\t", header=True, schema=name_basics_schema)

df_akas = spark.read.csv(imdb_path + "title.akas.tsv", sep="\t", header=True, schema=title_akas_schema)

df_title = spark.read.csv(imdb_path + "title.basics.tsv", sep="\t", header=True, schema=title_schema)

df_crew = spark.read.csv(imdb_path + "title.crew.tsv", sep="\t", header=True, schema=title_crew_schema)

df_episode = spark.read.csv(imdb_path + "title.episode.tsv", sep="\t", header=True, schema=title_episode_schema)

df_principals = spark.read.csv(imdb_path + "title.principals.tsv", sep="\t", header=True, schema=title_principals_schema)

df_ratings = spark.read.csv(imdb_path + "title.ratings.tsv", sep="\t", header=True, schema=title_ratings_schema)

In [15]:
df_title.show(5)
df_title.printSchema()
print("Кількість рядків:", df_title.count())
print("Кількість стовпців:", len(df_title.columns))
df_title.select("isAdult", "runtimeMinutes").describe().show(20)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1|              

In [16]:
df_people.show(5)
df_people.printSchema()
print("Кількість рядків:", df_people.count())
print("Кількість стовпців:", len(df_people.columns))
df_people.select("birthYear", "deathYear").describe().show(20)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0072308,tt00504...|
|nm0000002|  Lauren Bacall|     1924|     2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|Brigitte Bardot|     1934|     NULL|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00694...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- deathYear: integer (nullable = t

In [17]:
df_akas.show(5)
df_akas.printSchema()
print("Кількість рядків:", df_akas.count())
print("Кількість стовпців:", len(df_akas.columns))
df_akas.select("isOriginalTitle").describe().show(20)

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Carmencita|    \N|      \N|   original|           \N|              1|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|literal title|              0|
|tt0000001|       3|          Carmencita|    US|      \N|imdbDisplay|           \N|              0|
|tt0000001|       4|Carmencita - span...|    HU|      \N|imdbDisplay|           \N|              0|
|tt0000001|       5|          Καρμενσίτα|    GR|      \N|imdbDisplay|           \N|              0|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
only showing top 5 rows

root
 |-- titleId: string (nullable = true)
 |-- ordering: integer (nullabl

In [19]:
df_crew.show(5)
df_crew.printSchema()
print("Кількість рядків:", df_crew.count())
print("Кількість стовпців:", len(df_crew.columns))
df_crew.select("directors", "writers").describe().show(20)

+---------+---------+---------+
|   tconst|directors|  writers|
+---------+---------+---------+
|tt0000001|nm0005690|       \N|
|tt0000002|nm0721526|       \N|
|tt0000003|nm0721526|nm0721526|
|tt0000004|nm0721526|       \N|
|tt0000005|nm0005690|       \N|
+---------+---------+---------+
only showing top 5 rows

root
 |-- tconst: string (nullable = true)
 |-- directors: string (nullable = true)
 |-- writers: string (nullable = true)

Кількість рядків: 11595616
Кількість стовпців: 3
+-------+---------+--------------------+
|summary|directors|             writers|
+-------+---------+--------------------+
|  count| 11595616|            11595616|
|   mean|     NULL|                NULL|
| stddev|     NULL|                NULL|
|    min|       \N|                  \N|
|    max|nm9993709|nm9993713,nm54113...|
+-------+---------+--------------------+



In [21]:
df_episode.show(5)
df_episode.printSchema()
print("Кількість рядків:", df_episode.count())
print("Кількість стовпців:", len(df_episode.columns))
df_episode.select("seasonNumber", "episodeNumber").describe().show(20)

+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0031458|  tt32857063|        NULL|         NULL|
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
|tt0042889|   tt0989125|        NULL|         NULL|
|tt0043426|   tt0040051|           3|           42|
+---------+------------+------------+-------------+
only showing top 5 rows

root
 |-- tconst: string (nullable = true)
 |-- parentTconst: string (nullable = true)
 |-- seasonNumber: integer (nullable = true)
 |-- episodeNumber: integer (nullable = true)

Кількість рядків: 8919371
Кількість стовпців: 4
+-------+------------------+-----------------+
|summary|      seasonNumber|    episodeNumber|
+-------+------------------+-----------------+
|  count|           7094691|          7094691|
|   mean| 3.818385465977306|419.7858889132733|
| stddev|23.795991217886993|1330.196264186153

In [23]:
df_principals.show(5)
df_principals.printSchema()
print("Кількість рядків:", df_principals.count())
print("Кількість стовпців:", len(df_principals.columns))
df_principals.select("ordering").describe().show(20)

+---------+--------+---------+---------------+--------------------+----------+
|   tconst|ordering|   nconst|       category|                 job|characters|
+---------+--------+---------+---------------+--------------------+----------+
|tt0000001|       1|nm1588970|           self|                  \N|  ["Self"]|
|tt0000001|       2|nm0005690|       director|                  \N|        \N|
|tt0000001|       3|nm0005690|       producer|            producer|        \N|
|tt0000001|       4|nm0374658|cinematographer|director of photo...|        \N|
|tt0000002|       1|nm0721526|       director|                  \N|        \N|
+---------+--------+---------+---------------+--------------------+----------+
only showing top 5 rows

root
 |-- tconst: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)

Кількість рядків: 920472

In [24]:
df_ratings.show(5)
df_ratings.printSchema()
print("Кількість рядків:", df_ratings.count())
print("Кількість стовпців:", len(df_ratings.columns))
df_ratings.select("averageRating", "numVotes").describe().show(20)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2149|
|tt0000002|          5.5|     292|
|tt0000003|          6.5|    2185|
|tt0000004|          5.3|     188|
|tt0000005|          6.2|    2931|
+---------+-------------+--------+
only showing top 5 rows

root
 |-- tconst: string (nullable = true)
 |-- averageRating: float (nullable = true)
 |-- numVotes: integer (nullable = true)

Кількість рядків: 1560001
Кількість стовпців: 3
+-------+----------------+------------------+
|summary|   averageRating|          numVotes|
+-------+----------------+------------------+
|  count|         1560001|           1560001|
|   mean|6.95038644828358|1022.6708694417504|
| stddev|1.38674558917145|17816.708925974348|
|    min|             1.0|                 5|
|    max|            10.0|           3032380|
+-------+----------------+------------------+



#**БІЗНЕС ПИТАННЯ**

In [26]:
#фільми українською
#FILTER
ukr_movies = df_akas.filter(col("language") == 'uk').select("title").distinct()
ukr_movies.show(20)

+--------------------+
|               title|
+--------------------+
|             Я люблю|
|          Знак Зорро|
|               Друга|
|  Zacharovane mistse|
|       Кіра Кіраліна|
|         Чорна шкіра|
|Чи легко бути мол...|
|           Студентка|
|Народжена революцією|
|              Привид|
|    Bahata narechena|
|    Сумка дипкур'єра|
|     Люлька комунара|
|    Жінка в дзеркалі|
|      Дім зненависти|
|     U mertviy petli|
|       П'ять вечорів|
|          Ridna krov|
|        Їхнє царство|
|Кавказька полонян...|
+--------------------+
only showing top 20 rows



In [31]:
#топ 5 фільмів кожною мовою
#WINDOW and JOIN
df_akas_with_ratings = df_akas.join(df_ratings, df_akas.titleId == df_ratings.tconst, "inner")
window = Window.partitionBy("language").orderBy(col("averageRating").desc())
top_5_lang = df_akas_with_ratings.withColumn("rank", rank().over(window)).filter(col("rank") <= 5)
top_5_lang.show(20)

+----------+--------+--------------------+------+--------+-----------+--------------+---------------+----------+-------------+--------+----+
|   titleId|ordering|               title|region|language|      types|    attributes|isOriginalTitle|    tconst|averageRating|numVotes|rank|
+----------+--------+--------------------+------+--------+-----------+--------------+---------------+----------+-------------+--------+----+
| tt3984412|       1|I'm Not Going to ...|  NULL|    NULL|       NULL|          NULL|           NULL| tt3984412|          8.1|       7|   1|
| tt3984412|       2|I'm Not Going to ...|  NULL|    NULL|       NULL|          NULL|           NULL| tt3984412|          8.1|       7|   1|
|tt12149332|       1|Jeopardy! College...|  NULL|    NULL|       NULL|          NULL|           NULL|tt12149332|          6.9|       9|   3|
|tt12149332|       2|Jeopardy! College...|  NULL|    NULL|       NULL|          NULL|           NULL|tt12149332|          6.9|       9|   3|
|tt28535095| 

In [32]:
#фільми після 2015
#FILTER
after_2015_movies = df_title.filter(col("startYear") > '2015').select("primaryTitle", "startYear")
after_2015_movies.show(20)

+--------------------+---------+
|        primaryTitle|startYear|
+--------------------+---------+
|    Tötet nicht mehr|     2019|
|            Aufsätze|     2021|
|Number 14: Late S...|     2023|
|            EMS nr 1|     2016|
|A Embalagem de Vidro|     2020|
|The Tango of the ...|     2020|
|The Other Side of...|     2018|
|   Socialist Realism|     2023|
|       Anything Goes|     2022|
|Histórias de Comb...|     2022|
| Grizzly II: Revenge|     2020|
|      Loading Ludwig|     2022|
|The Wandering Soa...|     2017|
|Beach Birds for C...|     2024|
|  Neues in Wittstock|     2021|
|       Fado Lusitano|     2023|
|         A Thin Life|     2018|
|             Bigfoot|     2017|
| Mariette in Ecstasy|     2019|
|The Surgeon of th...|     2022|
+--------------------+---------+
only showing top 20 rows



In [33]:
#фільми довші за 122 хв
#FILTER
more_122_min = df_title.filter(col("runtimeMinutes").cast("int") > 122).select("primaryTitle", "runtimeMinutes")
more_122_min.show(20)

+--------------------+--------------+
|        primaryTitle|runtimeMinutes|
+--------------------+--------------+
|What Happened to ...|           150|
|The Adventures of...|           300|
|Germinal; or, The...|           150|
|Les Misérables, P...|           300|
|The Active Life o...|           170|
|The Beloved Adven...|           450|
|             Cabiria|           148|
|  The Child of Paris|           124|
|The Exploits of E...|           220|
|The Hazards of Helen|          1428|
|Lucille Love: The...|           300|
|      The Master Key|           310|
|The Perils of Pau...|           199|
|The Port of Missi...|           139|
|El signo de la tribu|           219|
|  The Trey o' Hearts|           310|
|The Birth of a Na...|           195|
|       The Black Box|           195|
|     The Broken Coin|           440|
|El testamento de ...|           293|
+--------------------+--------------+
only showing top 20 rows



In [34]:
#кількість фільмів по жанрах
#GROUPBY
amount_movies_per_genre = df_title.groupBy("genres").count().orderBy("count", ascending=False)
amount_movies_per_genre.show(20)

+-----------------+-------+
|           genres|  count|
+-----------------+-------+
|            Drama|1307026|
|           Comedy| 754248|
|        Talk-Show| 733689|
|             News| 605240|
|      Documentary| 558333|
|    Drama,Romance| 525895|
|               \N| 510324|
|       Reality-TV| 369200|
|            Adult| 317417|
|   News,Talk-Show| 262821|
|            Short| 226072|
|      Drama,Short| 217471|
|           Family| 203691|
|        Game-Show| 190400|
|     Comedy,Short| 163789|
|Documentary,Short| 160922|
|            Sport| 148037|
|            Music| 120145|
|          Romance| 116949|
| Comedy,Talk-Show| 116383|
+-----------------+-------+
only showing top 20 rows



In [35]:
#Cередня тривалість фільмів по жанрах
#GROUPBY
avg_length_movies_per_genre = df_title.withColumn("runtime", col("runtimeMinutes").cast("int")) \
    .groupBy("genres").avg("runtime").orderBy("avg(runtime)", ascending=False)
avg_length_movies_per_genre.show(20)

+--------------------+------------------+
|              genres|      avg(runtime)|
+--------------------+------------------+
|Comedy,Drama,Real...|  7646.80206185567|
|Music,Musical,Rea...|           488.625|
|Action,Fantasy,Ta...|             480.0|
|        News,Romance|             480.0|
|Game-Show,History...|             225.0|
|Action,Adult,History|             200.0|
|Family,News,Reali...|             193.2|
|Action,Game-Show,...|             180.0|
|Adventure,Crime,News|             178.0|
|          Sci-Fi,War|162.16666666666666|
|Game-Show,Music,T...|159.80769230769232|
|Musical,Romance,T...|             159.4|
|Adult,Fantasy,Wes...|             158.0|
|Documentary,Fanta...|             157.0|
|Family,Musical,Re...|             155.0|
|Music,Sport,Talk-...|150.33333333333334|
|Biography,Music,S...|             150.0|
|Mystery,Reality-T...|             150.0|
|Adult,Drama,Reali...|             149.0|
|   Adventure,Fantasy|145.88201712654615|
+--------------------+------------

In [37]:
#топ3 фільми за рейтингом у кожному жанрі
#WINDOW
df_joined = df_title.join(df_ratings, "tconst")
windowSpec = Window.partitionBy("genres").orderBy(col("averageRating").desc())
top_3_each_genre = df_joined.withColumn("rank", rank().over(windowSpec)).filter(col("rank") <= 3)
top_3_each_genre.select("primaryTitle", "genres", "averageRating", "rank").show(20)


+--------------------+------+-------------+----+
|        primaryTitle|genres|averageRating|rank|
+--------------------+------+-------------+----+
|"I'm Not Going to...|  NULL|          8.1|   1|
|"Jeopardy! Colleg...|  NULL|          6.9|   2|
|"I'm into Sharif ...|  NULL|          6.2|   3|
|Psychology of Con...|Action|         10.0|   1|
|In a Dying World ...|Action|         10.0|   1|
|       Episode #1.17|Action|         10.0|   1|
|       Bhasm Ka Sach|Action|         10.0|   1|
|            El bruto|Action|         10.0|   1|
|           Nachtflug|Action|         10.0|   1|
|Bhasm Ki Nayi Peh...|Action|         10.0|   1|
|               Pilot|Action|         10.0|   1|
|      Phoenix Harris|Action|         10.0|   1|
| Wasiat Naga Bergola|Action|         10.0|   1|
|El árbol de la horca|Action|         10.0|   1|
|Defensive Improvi...|Action|         10.0|   1|
|         Episódio 08|Action|         10.0|   1|
|               Pilot|Action|         10.0|   1|
|Testamento En Blanc

In [38]:
#актори і фільми
#JOIN
df_actors = df_principals.filter(col("category") == "actor").join(df_people, "nconst")
df_actors.select("primaryName", "tconst").show(5)

+------------+---------+
| primaryName|   tconst|
+------------+---------+
|John Belushi|tt0076816|
|John Belushi|tt0077147|
|John Belushi|tt0077621|
|John Belushi|tt0077975|
|John Belushi|tt0078723|
+------------+---------+
only showing top 5 rows



# **ЗАПИС РЕЗУЛЬТАТІВ**

In [44]:
top_3_each_genre.write.csv("/content/drive/MyDrive/imdb/output/top3_each_genre.csv", header=True, mode="overwrite")
print("Запис результатів завершено!")

Запис результатів завершено!


In [45]:
ukr_movies.write.csv("/content/drive/MyDrive/imdb/output/ukr_movies.csv", header=True, mode="overwrite")
print("Запис результатів завершено!")

Запис результатів завершено!


In [46]:
avg_length_movies_per_genre.write.csv("/content/drive/MyDrive/imdb/output/avg_length_movies_per_genre.csv", header=True, mode="overwrite")
print("Запис результатів завершено!")

Запис результатів завершено!


In [47]:
more_122_min.write.csv("/content/drive/MyDrive/imdb/output/more_122_min.csv", header=True, mode="overwrite")
print("Запис результатів завершено!")

Запис результатів завершено!


In [48]:
after_2015_movies.write.csv("/content/drive/MyDrive/imdb/output/after_2015_movies.csv", header=True, mode="overwrite")
print("Запис результатів завершено!")

Запис результатів завершено!


In [49]:
top_5_lang.write.csv("/content/drive/MyDrive/imdb/output/top_5_lang.csv", header=True, mode="overwrite")
print("Запис результатів завершено!")

Запис результатів завершено!


In [50]:
amount_movies_per_genre.write.csv("/content/drive/MyDrive/imdb/output/amount_movies_per_genre.csv", header=True, mode="overwrite")
print("Запис результатів завершено!")

Запис результатів завершено!
